In [1]:
# Load the gams extension
%load_ext gams_magic

# Simple SOCP problem

We would like to minimize the function 
$$f: \mathbb{R}^3 \mapsto \mathbb{R},$$
with values:
$$ f(x) = \max \left( x_1 + x_2 - \min ( \min ( x_1+2,x_2+2x_1-5),x_3 - 6),
 \frac{(x_1-x_3)^2 + 2x_2^2}{1-x_1} \right),
$$
with the constraint \$\| x \_{\infty} < 1 \$.   Explain precisely how to formulate the 
problem as an SOCP in standard form.
### Solve using GAMS.

In [2]:
%%gams

set i /1*3/;
set j /1*2/;

* set the precision level
scalar epsilon /0.00001/;

free variable x(i), w(j);
free variables
    inner_most "min(x1 + 2, x2 + 2 * x1 - 5)",
    inner "min(inner_most, x3 - 6)",
    f "max( x1 + x2 - inner, ((x1 - x3)^2 + 2 * x2^2) / (1 - x1) )";

positive variables
    y "((x1 - x3)^2 + 2 * x2^2) / (1 - x1)",
    t "1 - x1";

equations
    inner_most_constr1,
    inner_most_constr2,
    inner_constr1,
    inner_constr2,
    f_constr1,
    f_constr2,
    x_up_constr(i),
    x_low_constr(i),
    socp_eq,
    t_eq,
    w_eq1,
    w_eq2;

* Equation definitions
inner_most_constr1..
    inner_most =l= x("1") + 2;

inner_most_constr2..
    inner_most =l= x("2") + 2 * x("1") - 5;

inner_constr1..
    inner =l= inner_most;

inner_constr2..
    inner =l= x("3") - 6;

f_constr1..
    f =g= x("1") + x("2") - inner;

f_constr2..
    f =g= y;

x_up_constr(i)..
    x(i) =l= 1 - epsilon;

x_low_constr(i)..
    x(i) =g= -1 + epsilon;

socp_eq..    
    y*t =g= sum(j, sqr(w(j)));

t_eq..
    t =e= 1 - x("1");

w_eq1..
    w("1") =e= x("1") - x("3");

w_eq2..
    w("2") =e= sqrt(2) * x("2");

model socp /all/;
option qcp = cplexd;
solve socp using qcp minimizing f;
display x.l;

,Solver Status,Model Status,Objective,#equ,#var,Model Type,Solver,Solver Time
0,Normal (1),Optimal Global (1),4.5,16,10,QCP,CPLEXD,0.022
